# Baseball Lab 10


Welcome to the tenth, and final, baseball lab! This lab is due by  **Monday May 3rd at 11:30pm**. 

As always, you are welcome to work with others on the lab, but if you do work with others after class, please note on the reflection quiz who you worked with. If you have questions about the homework please use [Ed discussions](https://edstem.org/us/courses/4202/discussion/) so that everyone can benefit from your question. Additionally, please answer questions others raise on Ed discussions. Finally, please feel free to attend Neel's and my office hours for additional help. 


#### Today's Baseball Lab

In today's exercises, you'll get practice doing:

1. Examining the concept of *regression to the mean* 
2. See how to fit more complicated (non-linear) functions to data 
3. Explore Bill James "Pythagorean Expectation" and discuss cross-validation

Reading that will be useful for solving the problems: class textbook [chapter 15](https://www.inferentialthinking.com/chapters/15/Prediction.html) 

The code below loads the package we will need, and defines the `add_derived_stats()` and `get_rmse()` functions from lab 9.


In [1]:
# importing packages that will be used in this lab

from datascience import *
import pandas as pd
import numpy as np
from scipy.stats import binom
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf 


# ignore invalid division warnings
np.seterr(divide='ignore', invalid='ignore')


np.random.seed(173)

%matplotlib inline



def get_rmse(the_data, y_col, x_col):
    
    the_formula = y_col + '~' + x_col
    
    lm = smf.ols(the_formula, data = the_data).fit()

    predictions = lm.predict(the_data).to_numpy()
   
    return (np.mean((the_data[y_col] - predictions)**2))**.5





# a function that adds that statistics: BA, SLG, OBP, and OPS
def add_derived_stats(teams_data):
    
    # batting average: ba
    ba = teams_data['H']/teams_data['AB']

    # create slugging percentage
    x1b = teams_data['H'] - (teams_data['2B'] + teams_data['3B'] + teams_data['HR'])
    slg = (x1b + (2 * teams_data['2B'] + 3 * teams_data['3B'] + 4 * teams_data['HR']))/teams_data['AB']

    # create plate appearance
    pa = teams_data['AB'] + teams_data['BB'] + teams_data['HBP'] + teams_data['SF']
    
    # create on-base percentage
    obp = (teams_data['H'] + teams_data['BB'] + teams_data['HBP'])/(teams_data['AB'] + teams_data['BB'] + teams_data['HBP'] + teams_data['SF'])

    # create on-base plus slugging percentage
    ops = obp + slg

    # create the table that has all the desired statistics
    teams_with_addition_stats = teams_data.with_column('1B', x1b).with_column('BA', ba).with_column('SLG', slg).with_column('OBP', obp).with_column('OPS', ops).with_column('PA', pa)

    return teams_with_addition_stats


# # If you are using Pandas you can use this code instead of the code above

# # a function that adds that statistics: BA, SLG, OBP, and OPS
# def add_derived_stats(teams_data):
    
#     # batting average: ba
#     ba = teams_data['H']/teams_data['AB']

#     # create slugging percentage
#     x1b = teams_data['H'] - (teams_data['2B'] + teams_data['3B'] + teams_data['HR'])
#     slg = (x1b + (2 * teams_data['2B'] + 3 * teams_data['3B'] + 4 * teams_data['HR']))/teams_data['AB']

#     # create on-base percentage
#     obp = (teams_data['H'] + teams_data['BB'] + teams_data['HBP'])/(teams_data['AB'] + teams_data['BB'] + teams_data['HBP'] + teams_data['SF'])

#     # create on-base plus slugging percentage
#     ops = obp + slg

#     # create the table that has all the desired statistics
#     teams_with_addition_stats = teams_data.copy()
#     teams_with_addition_stats["1B"] = x1b
#     teams_with_addition_stats["BA"] = ba
#     teams_with_addition_stats["SLG"] = slg
#     teams_with_addition_stats["OBP"] = obp
#     teams_with_addition_stats["OPS"] = ops
    
#     return teams_with_addition_stats


# Part 1: Regression to the mean - explaining the Rookie of the Year Curse


As discussed in the class textbook and on [wikipedia](https://en.wikipedia.org/wiki/Regression_toward_the_mean), "Sir Francis Galton observed that extreme characteristics (e.g., height) in parents are not passed on completely to their offspring. Rather, the characteristics in the offspring regress towards a mediocre point (a point which has since been identified as the mean)." In fact, Galton first coined the term "regression", for predicting a y value based on x values, based on the fact that the heights of children of tall (and short) parents tend to regress toward the average height of the population. 

A similar effect can be seen in baseball. For example, there is the well-known [Sports Illustrated Cover Jinx](https://en.wikipedia.org/wiki/Sports_Illustrated_cover_jinx), where players tend to perform worse after they appear on the cover Sports Illustrated Magazine. There is also the "Rookie of the Year Curse" where players tend to perform worse after receiving the Rookie of the Year award. In the following exercises, we will examine this phenomenon to understand what is driving it. 

<img src="https://upload.wikimedia.org/wikipedia/en/3/34/Sports_Illustrated_cover_January_21_2002.jpg">



## 1.1: Examining regression to the mean by simulating two seasons 

To understand the regression to the mean phenomenon, we will simulate the **performance** (statistics) of a set of baseball players based on their underlying **abilities** (parameters). 

The function written below called `simulate_players_stats(batting_abilities, n)` simulates players' performance statistics, based on their underlying abilities. The arguments to this function are: 

1. `batting_abilities`: an ndarray of proportions ($\pi$'s) specifying the abilities of a set of players. For example, it could be a 50 element ndarray of proportions specifying the OBP abilities of 50 players. Note: we would never know these real abilities perfectly in the "real world" but for the sake of simulation we can specify them perfectly. 
    
2. `n`: this lists the number of events to simulate for each player. For example, if we simulate OBP, then n would specify how many plate appearances we want to simulate for each player.

The `simulate_players_stats()` function returns simulated performance statistics ($\hat{p}$'s) from these players for n events. 

Note: I am sure you could easily write the `simulate_players_stats()` function yourself, but to give you more time to work on your final project I have written it for you :)



**Exercise 1.1 (16 points)**:  Let's use the `simulate_players_stats()` to simulate the player's observed batting averages (BA), based on their true batting average abilities. We will do this simulation for two seasons and see if we observe the regression to the mean phenomenon, e.g., players who have high BA's in the first season tend to have lower BAs in the second season, etc. 

To do this simulation, please complete the following steps: 

1. use the np.arange() to create an ndarray called `batting_abilities` that has players' true BA abilities. The abilities should range from .250 to .320 in steps of .001, i.e, this ndarray should list the abilities of a total of 70 players. 

2. Use the `simulate_players_stats()` function to create the observed performances for one season of 70 players based on the abilities of these 70 players as specified in the `batting_abilities` ndarray. Use n = 500 plate appearances for these players and save the results to the name `season_1`.

3. Repeat step 2 to simulate the BA performance for a second season and save it to the name `season_2`

4. Create a table called `ba_table` that has two columns called `Season 1` and `Season 2` which has the data for the simulated seasons. Then create a scatter plot of the two seasons from this table and add the regression line to this scatter plot using the `fit_line = True` argument.

5. Add the identity line (line of slope 1, intercept of 0) that shows how well players would do if they performed equally well on both seasons. To do this you can create a ndarray of abilities ranging from .200 to .380 at increments of .001 and then use the `plt.plot(x, x)` function to create this line. 


Does this plot show the regression to the mean phenomenon? In the answer section below explain why this is happening. 

**Answer:**




In [2]:

def simulate_players_stats(batting_abilities, n):
    
    all_stats = []
    
    for i in np.arange(1, len(batting_abilities)):
        
        prob_heads = batting_abilities[i]
        
        curr_stat = sample_proportions(n, [prob_heads, 1 - prob_heads]).item(0)
        
        all_stats.append(curr_stat)
        
    return all_stats



# create the simulated scatter for two seasons below













## 1.2 Bonus problem: Does the Rookie of the Year Curse exist? 

**Note: I am making this problem optional since I want to give you more time to work on your final project. We might try to do this in class, and if you feel inspired you can complete it and turn it in.** 

Above we showed that regression to the mean exists in a simple simulation, but does it actually occur in the real baseball data? While many people claim it does, can we actually see it in the data? Since it is going to be hard to find all the covers of Sports Illustrated that players were on, let's focus on the "Rookie of the Year Curse" to see if we can see in there.

**Exercise 1.2 (0 points)**: The code below loads a table of the list of awards players have won in their careers and also the season level batting data from all players. Try to create a scatter plot that has the following features: 

- For all players who won the rookie of the year award, create a scatter plot where the x-axis has their rookie season OPS statistic, and the y-axis has their second season OPS statistic. Only plot players who have OPS that are greater than 0.6 for both their rookie and second-year seasons (this will get rid of missing data and pitchers, etc.). Add the identity line to this plot as well to get a sense of players who performed worse their second season.

Also, in the answer section below, report the percentage of players who won rookie of the year award and had a worse OPS statistic in their second year (again, only use players who have OPS that are greater than 0.6 for both their rookie and second-year seasons).

**Answer:**







In [3]:
# read in a table of the awards and batting statistics
awards_url = "https://raw.githubusercontent.com/chadwickbureau/baseballdatabank/master/core/AwardsPlayers.csv"
award_table = Table.read_table(awards_url)

batting = Table.read_table('https://raw.githubusercontent.com/emeyers/SDS173/master/data/Batting.csv')  
batting = add_derived_stats(batting)


# create a scatter plot showing rookie season OPS vs. second season OPS 
#  for players who won the rookie of the year award

 
 













# Part 2: Non-linear functions of the predictors

To improve the fit of linear models to the data, we can also include additional predictors that are derived from the original predictions in our data set. For example, we might want to predict the amount of runs a team will score as a function of the number of home runs and home runs squared, which would result in the equation: $\hat{r} =  \hat{\beta}_0 + \hat{\beta}_1 \cdot HR + \hat{\beta}_2 \cdot HR^2$. This function still creates a linear combination of predictors, but since some of the predictors are now non-linear functions of the original predictors, the final prediction function is non-linear in the original predictors. 



## 2.1: Quadratic fits for predicting maximum BA as a function of the year

Let's examine a quadratic fit for predicting the maximum batting average in a season as a function of the year. 

**Exercise 2.1 (19 points)**: Please complete the following steps:

1. Create a table called `ba_max_year` that has maximum batting average for each year from 1920 to the present for all players who had more than 502 at-bats. Hint: the `tb.group()` method will be useful here. Also, relabel the column that is `BA max` to be simple `BA`. 

2. Add a column to the `ba_max_year` table called `year2` that has the values of the years squared. 

3. Fit a linear model predicting the batting average as a function of the year and save it to the name `fit_lin`. Also print out the coefficients of this linear model. 

4. Create a scatter plot of the maximum BA as a function of the year. Add to this scatter plot, a line for the predictions made by the linear model. Hint: the `fit.predict()` method will be useful here.

5. Repeat steps 3 and 4 but also add the quadratic year term and save the model to the name `fit_quad`. Again, print the coefficients and add a line for the quadratic fit. 

6. Use the `get_rmse()` function to get the RMSE for both models. In the answer section below, describe which model you think has the better fit. 

**Answer:**





In [4]:
batting = Table.read_table('https://raw.githubusercontent.com/emeyers/SDS173/master/data/Batting.csv')  
batting = add_derived_stats(batting)





















## 2.2: Adding higher-order polynomial terms

Let's now extend the model to include higher order terms up to degree 6.

**Exercise 2.2 (11 points)**: Please create a degree 6 model of the form: $\hat{BA} =  \hat{\beta}_0 + \hat{\beta}_1 \cdot year + \hat{\beta}_2 \cdot year^2 + \hat{\beta}_3 \cdot year^3 ... + \hat{\beta}_6 \cdot year^6$. Again, print the coefficients of this model, create a scatter plot that contains the fitted line, and print the RMSE. In the answer section below, state whether you think this model is a better fit than the quadratic model. 

**Answer:**






# Part 3: Bill James "Pythagorean expectation" and cross-validation


As we have previously discussed, one of the founders of [sabermetrics](https://en.wikipedia.org/wiki/Sabermetrics) was [Bill James](https://en.wikipedia.org/wiki/Bill_James). One of the many contributions Bill James made to the analysis of baseball data was his ["Pythagorean expectation"](https://en.wikipedia.org/wiki/Pythagorean_expectation) which predicts a team's win to loss ratio (W/L) at the end of a season as a function of the number of runs scored (R) and the number of rows allowed (RA) in a season. More specifically, the formula is: $\frac{W}{L} = (\frac{R}{RA})^2$
 
In these exercises we will examine the Pythagorean expectation formula and see if we can come up with a slightly better formula for predicting the win to loss ratio. We will also use cross-validation to make sure that the formula we come up with works better for predicting new data rather than just overfitting to the data we have. 


## 3.1: Assessing how well the Pythagorean expectation works

Let's start by examining how well the Pythagorean expectation formula works for predicting the win to loss ratio. 


**Exercise 3.1 (16 points)**:  The code below loads data for teams since 1970 that have played 162 games. Please evaluate how well the Pythagorean expectation formula works by completing the following steps: 

1. Add a column to the `teams_162` table called 'WL_ratio' that has the win loss ratio for all teams.

2. Add a column to the `teams_162` table called 'Pythag_prediction' that has the Pythagorean expectation predicted value for all teams; i.e., that has the value $(R/RA)^2$.

3. Create a scatter plot of win loss ratio as a function of the Pythagorean expectation value and add a regression line to the plot. 

4. Fit a least squares model to predict the loss ratio as a function of the Pythagorean expectation and print out the slope and intercept found.  In the answer section below, report what the slope and intercept should be if the Pythagorean expectation was a perfect fit for the data.

5. Finally print the RMSE for that the Pythagorean expectation formula gives using the `get_rmse()` function.


**Answer**




In [5]:
teams = Table.read_table('https://raw.githubusercontent.com/emeyers/SDS173/master/data/Teams.csv')  
















## 3.2:  Improving the Pythagorean expectation formula

Let's now see if we can improve on the Pythagorean expectation formula. In particular, the exponent on the runs to runs allowed ratio (R/RA) is 2; i.e., the formula is $\frac{W}{L} = (\frac{R}{RA})^2$. But perhaps there is a better exponent that should be used rather than 2; i.e., perhaps there is a value $k$ that would give a better fit in the formula: $\frac{W}{L} = (\frac{R}{RA})^k$. 

To find the value $k$ that works best, we can take the log of the general Pythagorean expectation formula: $log(\frac{W}{L}) = log((\frac{R}{RA})^k) = k \cdot log(\frac{R}{RA})$. We can then fit a linear model for $y = log(\frac{W}{L})$ as a function of $x = log(\frac{R}{RA})$ to find the optimum value for $k$. 

We should note that in fitting the above model there is no intercept term. We can fit a model without an intercept term using the statsmodels formula interface by including a -1 in our formula. i.e., using the syntax: `smf.ols('y ~ x -1', data = my_data)`.

**Exercise 3.2 (16 points)**: Let's try to find the optimal exponent *k* by completing the following steps: 

1. Create a new table called `teams_162_transform` that starts with the `teams_162` table and adds a column called 'log_WL_ratio' which has the log of the wins to losses

2. Add a column 'log_R_RA_ratio' to the `teams_162_transform` table which is the log of R/RA. 

3. Fit a linear regression model for predicting the 'log_WL_ratio' as a function of 'log_R_RA_ratio'. Make sure the model only has a slope and no intercept. Print the value of the slope for this model. 

4. Add a column called 'better_Pythag_prediction' to the `teams_162_transform` table that contains the predictions for the new linear regression model using the "optimal" *k* found in step 3. 

5. Get the RMSE from using the model with the optimum *k*. Is the RMSE smaller with the optimal k? 

Report what the optimal k is, what the RMSE is with this optimal k, and what the RMSE is with the original exponent of 2.

**Answer**





## 3.3: Using cross-validation to assess if the model with the optimal k really makes better predictions

In exercise 3.2 we found the "optimal k" and assessed how well the model fit (via the RMSE) using the same data. Using the same data to fit and assess a model is generally a bad idea because this can lead to "overfitting" where the model is really just fitting the noise in the data rather than a true underlying relationship. 

**Exercise 3.3 (19 points)**:   To assess whether the new model really is better at making predictions, we can use cross-validation where we fit the model on one subset of data and then evaluate how well the model makes predictions (via the RMSE) on a new data set. Let's try this now by fitting the model on data from 2000 to 2009 and making predictions on data from 2010 to 2018 using the following steps: 

1. Create a data table called `data_2000s` from the `teams_162_transform` table that contains only data from 2000 to 2009.

2. Create a data table called `data_2010s` from the `teams_162_transform` table that contains only data from 2010 to 2018.

3. Print the RMSE for the data from 2010's using the Pythagorean expectation formula (i.e., with the exponent of 2). 

4. Fit a model of log_WL_ratio as a function of log_R_RA_ratio using data from only 2000's. Print the value for the exponent for this model.

5. Add a column called 'better_Pythag_prediction2' to the `teams_162_transform` table that has the predictions based on using the coefficient *k* found from using the data from 2000's (in step 4). 

6. Get the RMSE for the log_WL_ratio in 2010's based on the better_Pythag_prediction2 column. In the answer section below, report whether the RMSE for the model fit with the optimal $k$ found using data from the 2000's makes better predictions on the data from the 2010's compared to using the default Pythagorean expectation formula that has $k$ = 2. 

**Answers**





# Part 4: Reflection (3 points)

How did this lab go? Please complete the [reflection homework 10](https://yale.instructure.com/courses/65116/quizzes/35185) which is under the quiz section on Canvas to let us know how it went and to reflect on what you learned. 
